In [45]:
#Start work on Dota winning preditcion
import os
import pandas as pd
import numpy as np 
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import cross_val_score , GridSearchCV
from sklearn.linear_model import LogisticRegression
import json
from tqdm import tqdm_notebook
import lightgbm as lgb

PATH_TO_DATA = '../input'

#Train and test set
df_train_features = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_features.csv'), index_col='match_id_hash')
df_test_features = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_features.csv'), index_col='match_id_hash')

#Target variable 
df_train_targets = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_targets.csv'), 
                                   index_col='match_id_hash')

df_train_features.shape

(39675, 245)

Let's check how our train/target data looks like.

In [46]:
df_train_targets.head()

,game_time,radiant_win,duration,time_remaining,next_roshan_team
match_id_hash,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,False,992,837,NaN
b9c57c450ce74a2af79c9ce96fac144d,658,True,1154,496,NaN
6db558535151ea18ca70a6892197db41,21,True,1503,1482,Radiant
46a0ddce8f7ed2a8d9bd5edcbb925682,576,True,1952,1376,NaN
b1b35ff97723d9b7ade1c9c3cf48f770,453,False,2001,1548,NaN


In [47]:
df_train_features.head()

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,r1_gold,r1_lh,r1_xp,r1_health,r1_max_health,r1_max_mana,r1_level,r1_x,r1_y,r1_stuns,r1_creeps_stacked,r1_camps_stacked,r1_rune_pickups,r1_firstblood_claimed,r1_teamfight_participation,r1_towers_killed,r1_roshans_killed,r1_obs_placed,r1_sen_placed,r2_hero_id,r2_kills,r2_deaths,r2_assists,r2_denies,r2_gold,r2_lh,r2_xp,r2_health,r2_max_health,r2_max_mana,...,d4_health,d4_max_health,d4_max_mana,d4_level,d4_x,d4_y,d4_stuns,d4_creeps_stacked,d4_camps_stacked,d4_rune_pickups,d4_firstblood_claimed,d4_teamfight_participation,d4_towers_killed,d4_roshans_killed,d4_obs_placed,d4_sen_placed,d5_hero_id,d5_kills,d5_deaths,d5_assists,d5_denies,d5_gold,d5_lh,d5_xp,d5_health,d5_max_health,d5_max_mana,d5_level,d5_x,d5_y,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,7,1,11,11,0,0,0,0,543,7,533,358,600,350.93784,2,116,122,0.000000,0,0,1,0,0.000000,0,0,0,0,78,0,0,0,3,399,4,478,636,720,254.93774,...,760,760,326.93780,2,90,150,0.000000,0,0,2,1,1.0,0,0,1,0,34,0,0,0,0,851,11,870,593,680,566.93805,3,128,128,0.000000,0,0,0,0,0.00,0,0,0,0
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,15,7,2,0,7,5257,52,3937,1160,1160,566.93805,8,76,78,0.000000,0,0,0,0,0.437500,0,0,0,0,96,3,1,2,3,3394,19,3897,1352,1380,386.93787,...,567,1160,410.93790,6,124,142,0.000000,0,0,6,0,0.5,0,0,0,0,92,0,2,0,1,1423,8,1136,800,800,446.93793,4,180,176,0.000000,0,0,0,0,0.00,0,0,0,0
6db558535151ea18ca70a6892197db41,21,23,0,0,0,101,0,0,0,0,176,0,0,680,680,506.93800,1,118,118,0.000000,0,0,0,0,0.000000,0,0,0,0,51,0,0,0,0,176,0,0,720,720,278.93777,...,600,600,302.93777,1,176,110,0.000000,0,0,0,0,0.0,0,0,0,0,17,0,0,0,0,96,0,0,640,640,446.93793,1,162,162,0.000000,0,0,0,0,0.00,0,0,0,0
46a0ddce8f7ed2a8d9bd5edcbb925682,576,22,7,1,4,14,1,0,3,1,1613,0,1471,900,900,290.93777,4,170,96,2.366089,0,0,5,0,0.571429,0,0,0,0,99,1,0,1,2,2816,30,3602,878,1100,494.93796,...,1160,1160,386.93787,4,176,100,4.998863,0,0,2,0,0.0,0,0,0,0,86,0,1,0,1,1333,2,1878,630,740,518.93800,5,82,160,8.664527,3,1,3,0,0.00,0,0,2,0
b1b35ff97723d9b7ade1c9c3cf48f770,453,22,7,1,3,42,0,1,1,0,1404,9,1351,1000,1000,338.93784,4,80,164,9.930903,0,0,4,0,0.500000,0,0,0,0,69,1,0,0,0,1840,14,1693,868,1000,350.93784,...,680,680,374.93787,4,176,108,13.596678,0,0,2,0,0.5,0,0,0,0,1,0,1,1,8,2199,32,1919,692,740,302.93777,5,104,162,0.000000,2,1,2,0,0.25,0,0,0,0


Load training and test data.                                                                                                                                                            There are basic functions provided by organizators plus some new variables taken from raw data : damage distributed across multiple categories, ability levels , buybakcs use etc. .

In [48]:
with open(os.path.join(PATH_TO_DATA, 'train_matches.jsonl')) as fin:
    # read the 18-th line
    for i in range(27):
        line = fin.readline()
    
    # read JSON into a Python object 
    match = json.loads(line)

In [49]:
def read_matches(matches_file):
    
    MATCHES_COUNT = {
        'test_matches.jsonl': 10000,
        'train_matches.jsonl': 39675,
    }
    _, filename = os.path.split(matches_file)
    total_matches = MATCHES_COUNT.get(filename)
    
    with open(matches_file) as fin:
        for line in tqdm_notebook(fin, total=total_matches):
            yield json.loads(line)

In [50]:
import collections

MATCH_FEATURES = [
    ('game_time', lambda m: m['game_time']),
    ('game_mode', lambda m: m['game_mode']),
    ('lobby_type', lambda m: m['lobby_type']),
    ('objectives_len', lambda m: len(m['objectives'])),
    ('chat_len', lambda m: len(m['chat'])),
    ('number_of_teamfights', lambda m: len(m['teamfights'])),
]

PLAYER_FIELDS = [
    'hero_id',
    
    'kills',
    'deaths',
    'assists',
    'denies',
    
    'gold',
    'lh',
    'xp',
    'health',
    'max_health',
    'max_mana',
    'level',

    'x',
    'y',
    
    'stuns',
    'creeps_stacked',
    'camps_stacked',
    'rune_pickups',
    'firstblood_claimed',
    'teamfight_participation',
    'towers_killed',
    'roshans_killed',
    'obs_placed',
    'sen_placed',
]



def extract_features_csv(match):
    row = [
        ('match_id_hash', match['match_id_hash']),
    ]
    
    for field, f in MATCH_FEATURES:
        row.append((field, f(match)))
        
    for slot, player in enumerate(match['players']):
        if slot < 5:
            player_name = 'r%d' % (slot + 1)
        else:
            player_name = 'd%d' % (slot - 4)

        for field in PLAYER_FIELDS:
            column_name = '%s_%s' % (player_name, field)
            row.append((column_name, player[field]))
            
        row.append((player_name +'_ability_level', len(player['ability_upgrades'])))
        row.append((player_name +'_max_hero_hit', player['max_hero_hit']['value']))
        row.append((player_name +'_purchase_count', len(player['purchase_log'])))
        
        row.append((player_name +'_buyback', len(player['buyback_log'])))
        row.append((player_name +'_nearby_creep_death_count', player['nearby_creep_death_count']))
    
        
        type_of_damage = player['damage'].keys()
        hero_damage_keys = [key for key in type_of_damage if "hero" in key] 
        hero_damage_values = [player['damage'][key] for key in hero_damage_keys]
        
        nonhero_damage_keys = [key for key in type_of_damage if "creep" in key] 
        nonhero_damage_values = [player['damage'][key] for key in nonhero_damage_keys]
        
        neutral_damage_keys = [key for key in type_of_damage if "neutral" in key] 
        neutral_damage_values = [player['damage'][key] for key in nonhero_damage_keys]
        
        #Damage dealed 
        row.append((player_name +'_damage_deal_toheroes', sum(hero_damage_values)))
        row.append((player_name +'_damage_deal_tocreeps', sum(nonhero_damage_values)))
        row.append((player_name +'_damage_deal_toneutral', sum(neutral_damage_values)))
        
        #Items
        row.append((player_name +'_unique_items', len(player['item_uses'])))
        row.append((player_name +'_number_of_itmes_used', sum(player['item_uses'].values())))
        
        #Ability uses 
        row.append((player_name +'_unique_ability', len(player['ability_uses'])))
        row.append((player_name +'_number_of_ability_used', sum(player['ability_uses'].values())))
        
            
    return collections.OrderedDict(row)

def extract_targets_csv(match, targets):
    return collections.OrderedDict([('match_id_hash', match['match_id_hash'])] + [
        (field, targets[field])
        for field in ['game_time', 'radiant_win', 'duration', 'time_remaining', 'next_roshan_team']
    ])

In [51]:
%%time
df_new_features = []
df_new_targets = []

for match in read_matches(os.path.join(PATH_TO_DATA, 'train_matches.jsonl')):
    match_id_hash = match['match_id_hash']
    features = extract_features_csv(match)
    targets = extract_targets_csv(match, match['targets'])
    
    df_new_features.append(features)
    df_new_targets.append(targets)


CPU times: user 1min 42s, sys: 3.7 s, total: 1min 46s
Wall time: 1min 45s


In [52]:
df_new_features = pd.DataFrame.from_records(df_new_features).set_index('match_id_hash')
df_new_targets = pd.DataFrame.from_records(df_new_targets).set_index('match_id_hash')

Adding more features - think teamfights play some role in the game - so will add data about in different script.

In [53]:
#Teamfights

In [54]:

TEAMFIGHT_FIELDS = [
    
    'ability_uses',
    'item_uses',
    'killed',
    'deaths',
    'buybacks',
    'damage',
    'healing',
    'gold_delta',
    'xp_delta'
]

def extract_teamfight_features_csv(match):
    
    row = [
        ('match_id_hash', match['match_id_hash']),
    ]
    
    #Teamfights 
    
    for number,teamfight in enumerate(match["teamfights"]) :
        
        #row.append(("number_of_teamfights", len(match["teamfights"])))
        
        for slot,player in enumerate(teamfight["players"]) :
            
            if slot < 5:
                player_name = 'r%d' % (slot + 1) +"_teamfight_" + str(number)
            else:
                player_name = 'd%d' % (slot - 4) +"_teamfight_" + str(number)
                

            

            row.append((player_name +'_damage', player['damage']))
            row.append((player_name +'_gold_delta', player['gold_delta']))
            row.append((player_name +'_xp_delta', player['xp_delta']))

                
                
    return collections.OrderedDict(row)
                

In [55]:
%%time
df_train_teamfight_features = []


for match in read_matches(os.path.join(PATH_TO_DATA, 'train_matches.jsonl')):
    match_id_hash = match['match_id_hash']
    features = extract_teamfight_features_csv(match)
    
    df_train_teamfight_features.append(features)

df_train_teamfight_features = pd.DataFrame.from_records(df_train_teamfight_features).set_index('match_id_hash')
df_train_teamfight_features.head(5)


CPU times: user 1min 27s, sys: 2.33 s, total: 1min 29s
Wall time: 1min 28s


Loading test basic and test teamfight features in the similar way.

In [56]:
#Test  features

In [57]:
#Features
test_new_features = []
for match in read_matches(os.path.join(PATH_TO_DATA, 'test_matches.jsonl')):
    match_id_hash = match['match_id_hash']
    features = extract_features_csv(match)
    
    test_new_features.append(features)
test_new_features = pd.DataFrame.from_records(test_new_features).set_index('match_id_hash')

In [58]:
#Teamfight features
test_teamfight_new_features = []
for match in read_matches(os.path.join(PATH_TO_DATA, 'test_matches.jsonl')):
    match_id_hash = match['match_id_hash']
    features = extract_teamfight_features_csv(match)
    
    test_teamfight_new_features.append(features)
    
test_teamfight_new_features = pd.DataFrame.from_records(test_teamfight_new_features).set_index('match_id_hash')

Let's proceed with teamfight features first. Fill NA values below with 0.

In [59]:
#Work with teamfight features
df_train_teamfight_features.fillna(value = 0 , inplace = True)
test_teamfight_new_features.fillna(value = 0 , inplace = True)
test_teamfight_new_features.head()

,r1_teamfight_0_damage,r1_teamfight_0_gold_delta,r1_teamfight_0_xp_delta,r2_teamfight_0_damage,r2_teamfight_0_gold_delta,r2_teamfight_0_xp_delta,r3_teamfight_0_damage,r3_teamfight_0_gold_delta,r3_teamfight_0_xp_delta,r4_teamfight_0_damage,r4_teamfight_0_gold_delta,r4_teamfight_0_xp_delta,r5_teamfight_0_damage,r5_teamfight_0_gold_delta,r5_teamfight_0_xp_delta,d1_teamfight_0_damage,d1_teamfight_0_gold_delta,d1_teamfight_0_xp_delta,d2_teamfight_0_damage,d2_teamfight_0_gold_delta,d2_teamfight_0_xp_delta,d3_teamfight_0_damage,d3_teamfight_0_gold_delta,d3_teamfight_0_xp_delta,d4_teamfight_0_damage,d4_teamfight_0_gold_delta,d4_teamfight_0_xp_delta,d5_teamfight_0_damage,d5_teamfight_0_gold_delta,d5_teamfight_0_xp_delta,r1_teamfight_1_damage,r1_teamfight_1_gold_delta,r1_teamfight_1_xp_delta,r2_teamfight_1_damage,r2_teamfight_1_gold_delta,r2_teamfight_1_xp_delta,r3_teamfight_1_damage,r3_teamfight_1_gold_delta,r3_teamfight_1_xp_delta,r4_teamfight_1_damage,...,d2_teamfight_21_xp_delta,d3_teamfight_21_damage,d3_teamfight_21_gold_delta,d3_teamfight_21_xp_delta,d4_teamfight_21_damage,d4_teamfight_21_gold_delta,d4_teamfight_21_xp_delta,d5_teamfight_21_damage,d5_teamfight_21_gold_delta,d5_teamfight_21_xp_delta,r1_teamfight_22_damage,r1_teamfight_22_gold_delta,r1_teamfight_22_xp_delta,r2_teamfight_22_damage,r2_teamfight_22_gold_delta,r2_teamfight_22_xp_delta,r3_teamfight_22_damage,r3_teamfight_22_gold_delta,r3_teamfight_22_xp_delta,r4_teamfight_22_damage,r4_teamfight_22_gold_delta,r4_teamfight_22_xp_delta,r5_teamfight_22_damage,r5_teamfight_22_gold_delta,r5_teamfight_22_xp_delta,d1_teamfight_22_damage,d1_teamfight_22_gold_delta,d1_teamfight_22_xp_delta,d2_teamfight_22_damage,d2_teamfight_22_gold_delta,d2_teamfight_22_xp_delta,d3_teamfight_22_damage,d3_teamfight_22_gold_delta,d3_teamfight_22_xp_delta,d4_teamfight_22_damage,d4_teamfight_22_gold_delta,d4_teamfight_22_xp_delta,d5_teamfight_22_damage,d5_teamfight_22_gold_delta,d5_teamfight_22_xp_delta
match_id_hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
30cc2d778dca82f2edb568ce9b585caa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70e5ba30f367cea48793b9003fab9d38,739.0,458.0,107.0,526.0,128.0,50.0,0.0,0.0,0.0,505.0,-37.0,20.0,429.0,0.0,0.0,402.0,160.0,0.0,730.0,401.0,50.0,299.0,92.0,0.0,0.0,201.0,57.0,296.0,160.0,0.0,0.0,24.0,29.0,267.0,396.0,539.0,1272.0,866.0,1.0,379.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4d9ef74d3a2025d79e9423105fd73d41,276.0,194.0,95.0,190.0,101.0,238.0,99.0,134.0,268.0,532.0,266.0,238.0,382.0,79.0,183.0,37.0,79.0,86.0,0.0,-72.0,51.0,0.0,0.0,71.0,57.0,0.0,86.0,606.0,190.0,0.0,49.0,126.0,269.0,363.0,-95.0,28.0,0.0,182.0,268.0,722.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2bb79e0c1eaac1608e5a09c8e0c6a555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bec17f099b01d67edc82dfb5ce735a43,1201.0,259.0,511.0,762.0,41.0,572.0,190.0,287.0,362.0,531.0,165.0,224.0,233.0,-128.0,57.0,1002.0,-137.0,0.0,531.0,72.0,132.0,314.0,162.0,775.0,575.0,-122.0,0.0,1640.0,779.0,958.0,1203.0,-97.0,99.0,2320.0,1254.0,2408.0,1193.0,137.0,486.0,787.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

Below we calculate sum of same feature across all fights for both of the teams. The idea is to reduce number of features and present summary value across all players for every team. 

In [60]:
def sum_for_teamfight(data,stats,maxim ):
    df = data.copy()
    for feat_suff in stats:
            for team in 'r', 'd':
                players = [team +str(i) + "_teamfight_" + str(number_of_fight)  for i in range(1, 6) for number_of_fight in range(0,maxim)] # r1, r2...
                player_col_names = [player + "_" + feat_suff for player in players] # e.g. r1_gold, r2_gold
                
                df[team  + "_teamfight_" + feat_suff + "_sum"] = df[player_col_names].sum(axis=1) # e.g. r_gold_mean
                
                df.drop(columns=player_col_names, inplace=True) # remove raw features from the dataset
    
    return(df)

In [61]:
df_train_teamfight_features_average = sum_for_teamfight(df_train_teamfight_features,["damage","gold_delta","xp_delta"],maxim = 25)
test_teamfight_new_features_average = sum_for_teamfight(test_teamfight_new_features,["damage","gold_delta","xp_delta"],maxim = 23)

In [62]:
df_train_teamfight_features_average.head(5)

,r_teamfight_damage_sum,d_teamfight_damage_sum,r_teamfight_gold_delta_sum,d_teamfight_gold_delta_sum,r_teamfight_xp_delta_sum,d_teamfight_xp_delta_sum
match_id_hash,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,0.0,0.0,0.0,0.0,0.0,0.0
b9c57c450ce74a2af79c9ce96fac144d,4848.0,3081.0,2435.0,1696.0,2273.0,514.0
6db558535151ea18ca70a6892197db41,0.0,0.0,0.0,0.0,0.0,0.0
46a0ddce8f7ed2a8d9bd5edcbb925682,0.0,0.0,0.0,0.0,0.0,0.0
b1b35ff97723d9b7ade1c9c3cf48f770,0.0,0.0,0.0,0.0,0.0,0.0


There are a few records with "game_time" = 0 (which mean that statistic we have measured in time 0  of the game)  - not sure what they mean - looks like incorrect data. Will exclude this from datasets and create target feature.

In [63]:
#Create target feature

In [64]:
target = pd.DataFrame(df_new_targets[["radiant_win","time_remaining"]])
full_train = target.merge(df_new_features, on = "match_id_hash")
full_train["radiant_win"]  = full_train["radiant_win"].replace({True:1,False:0})
full_train = full_train.loc[full_train["game_time"]>0 , :]

df_new_features = full_train.drop(["radiant_win","time_remaining"],axis =  1)

target_variable = pd.DataFrame(full_train["radiant_win"].astype("int"))
y_train = full_train["radiant_win"].astype("int").values


In [65]:
target_variable.shape

(39655, 1)

The following is feature engineering step for our data :
* Combine combine r1_feature , r2_feature ... r5_feature into  one r_feature_sum column.
* For her0_id column we will use approach  similar to  OHE - i.e. for every id present hero_id as a column with 1/0 indicating if this hero present in the game or not.
* Create firstblood feature (radiant or dire).
* Create average per-time features - i.e. number of kills per minute by team.
* At the very end for all created features we have used ratio between corresponding radiant and dire feature.

In [66]:
numeric_features = ['deaths', 'assists', 'denies', 'health','gold','kills','xp' ,'x','y',
                    'max_health', 'max_mana', 'level', 'towers_killed', 'stuns', 'creeps_stacked', 
                    'camps_stacked', 'lh', 'rune_pickups',
                    'teamfight_participation', 'roshans_killed', 'obs_placed', 'sen_placed',
                    
                     'gold_per_minute','kills_per_minute','xp_per_minute','level_per_minute','damage_deal_tocreeps_per_minute',
                     "deaths_per_minute","number_of_ability_used_per_minute",
                    
                     "damage_deal_toheroes_per_minute","damage_deal_toneutral_per_minute",
                    
                    'ability_level','max_hero_hit','purchase_count','damage_deal_toheroes','damage_deal_tocreeps',
                    'unique_items','number_of_itmes_used','unique_ability','number_of_ability_used',
                   
                    "nearby_creep_death_count","buyback","damage_deal_toneutral"]

In [67]:
def combine_numeric_features (data, feature_suffixes):
    
    df = data.copy()
    for feat_suff in feature_suffixes:
        for team in 'r', 'd':
            players = [team +str(i) for i in range(1, 6)] # r1, r2...
            player_col_names = [player + "_" +feat_suff for player in players] # e.g. r1_gold, r2_gold
            
           
            df[team + "_" + feat_suff + "_"+ "sum"] = df[player_col_names].sum(axis=1) # e.g. r_gold_mean
            #df[team + "_" + feat_suff + "_"+ "min"] =  df[player_col_names].min(axis=1) # e.g. r_gold_min
            
            df.drop(columns=player_col_names, inplace=True) # remove raw features from the dataset
    return df

In [68]:
def hero_approach(df):
    for team in 'r', 'd':
        players = [team + str(i)  for i in range(1, 6)]
        hero_columns = [player + '_hero_id' for player in players]

        d = pd.get_dummies(df[hero_columns[0]])
        for c in hero_columns[1:]:
            d += pd.get_dummies(df[c])
        df = pd.concat([df, d.add_prefix(team  + '_hero_')], axis=1)
        df.drop(columns=hero_columns, inplace=True)
    return df

# df_train_features_heroes = hero_approach(df_new_features)
# df_test_features_heroes = hero_approach(test_new_features)


df_train_features_heroes = hero_approach(df_new_features)
df_test_features_heroes = hero_approach(test_new_features)

In [69]:
def firstblood(data) :
    df = data.copy()
    for team in 'r','d' :
        columns = [team + str(i) + '_firstblood_claimed' for i in range(1,6) ]
        print(columns)
        if team == 'r' :
            df["r_team_firstblood"] = df[columns].sum(axis = 1)
        df.drop(columns = columns,inplace = True)
    
    return(df)

In [70]:
df_train_features_firstblood = firstblood(df_train_features_heroes)
df_test_features_firstblood =  firstblood(df_test_features_heroes)

['r1_firstblood_claimed', 'r2_firstblood_claimed', 'r3_firstblood_claimed', 'r4_firstblood_claimed', 'r5_firstblood_claimed']
['d1_firstblood_claimed', 'd2_firstblood_claimed', 'd3_firstblood_claimed', 'd4_firstblood_claimed', 'd5_firstblood_claimed']
['r1_firstblood_claimed', 'r2_firstblood_claimed', 'r3_firstblood_claimed', 'r4_firstblood_claimed', 'r5_firstblood_claimed']
['d1_firstblood_claimed', 'd2_firstblood_claimed', 'd3_firstblood_claimed', 'd4_firstblood_claimed', 'd5_firstblood_claimed']


In [71]:

df_train_features_firstblood["game_minutes"] = (df_train_features_firstblood["game_time"]/float(60)).astype("float")
df_test_features_firstblood["game_minutes"] = (df_test_features_firstblood["game_time"]/float(60)).astype("float")

def add_new_features(data,original_features):
    
    df = data.copy()
    
    #Gold/min, Exp/min, and Kills/min
    
    for  feature in original_features :
        for team in "r","d" :
            players = [team +str(i) for i in range(1, 6)] # r1, r2...
            player_col_names = [player + "_" + feature for player in players] # e.g. r1_gold, r2_gold
            
            for player_feature in  player_col_names:
               # df[player_feature + '_per_minute'] = df[player_feature]/df["game_minutes"]
                 df.loc[df["game_minutes"] != 0 ,player_feature + '_per_minute'] = df[player_feature]/df["game_minutes"]
            
    return df

df_train_features_firstblood.head()

,game_time,game_mode,lobby_type,objectives_len,chat_len,number_of_teamfights,r1_kills,r1_deaths,r1_assists,r1_denies,r1_gold,r1_lh,r1_xp,r1_health,r1_max_health,r1_max_mana,r1_level,r1_x,r1_y,r1_stuns,r1_creeps_stacked,r1_camps_stacked,r1_rune_pickups,r1_teamfight_participation,r1_towers_killed,r1_roshans_killed,r1_obs_placed,r1_sen_placed,r1_ability_level,r1_max_hero_hit,r1_purchase_count,r1_buyback,r1_nearby_creep_death_count,r1_damage_deal_toheroes,r1_damage_deal_tocreeps,r1_damage_deal_toneutral,r1_unique_items,r1_number_of_itmes_used,r1_unique_ability,r1_number_of_ability_used,...,d_hero_79,d_hero_80,d_hero_81,d_hero_82,d_hero_83,d_hero_84,d_hero_85,d_hero_86,d_hero_87,d_hero_88,d_hero_89,d_hero_90,d_hero_91,d_hero_92,d_hero_93,d_hero_94,d_hero_95,d_hero_96,d_hero_97,d_hero_98,d_hero_99,d_hero_100,d_hero_101,d_hero_102,d_hero_103,d_hero_104,d_hero_105,d_hero_106,d_hero_107,d_hero_108,d_hero_109,d_hero_110,d_hero_111,d_hero_112,d_hero_113,d_hero_114,d_hero_119,d_hero_120,r_team_firstblood,game_minutes
match_id_hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,7,1,11,0,0,0,0,0,543,7,533,358,600,350.938,2,116,122,0.000,0,0,1,0.000,0,0,0,0,0,136,6,0,19,459,828,828,2,2,3,3,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.583333
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,2,7,2,0,7,5257,52,3937,1160,1160,566.938,8,76,78,0.000,0,0,0,0.438,0,0,0,0,6,216,28,0,79,4656,15306,15306,7,34,2,9,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10.966667
6db558535151ea18ca70a6892197db41,21,23,0,0,0,0,0,0,0,0,176,0,0,680,680,506.938,1,118,118,0.000,0,0,0,0.000,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.350000
46a0ddce8f7ed2a8d9bd5edcbb925682,576,22,7,1,4,0,1,0,3,1,1613,0,1471,900,900,290.938,4,170,96,2.366,0,0,5,0.571,0,0,0,0,2,182,17,0,59,3130,220,220,6,10,2,28,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,9.600000
b1b35ff97723d9b7ade1c9c3cf48f770,453,22,7,1,3,0,0,1,1,0,1404,9,1351,1000,1000,338.938,4,80,164,9.931,0,0,4,0.500,0,0,0,0,2,60,8,0,47,655,3892,3892,3,5,3,12,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,7.550000


In [72]:
def differences(data , feature_suff , stats) :
    
    df=data.copy()
    
    for col in feature_suff :
        for stat in stats :
            df[col +"_" + stat + "_ratio"] = df["r"  + "_" + col +"_" + stat] / df["d"  + "_" + col +"_" + stat]
                
            original_col = ["r"  + "_" + col +"_" + stat,"d"  + "_" + col +"_" + stat]
    
            df.drop(columns = original_col , inplace = True)
    return df

In [73]:
df_train_features_addtional_features = add_new_features(df_train_features_firstblood,
                                                        ['kills','gold','xp','level',"damage_deal_tocreeps",
                                                         "deaths","number_of_ability_used","damage_deal_toheroes","damage_deal_toneutral"])
df_test_features_addtional_features = add_new_features(df_test_features_firstblood,
                                                       ['kills','gold','xp','level',"damage_deal_tocreeps",
                                                        "deaths","number_of_ability_used","damage_deal_toheroes","damage_deal_toneutral"])

In [74]:
df_train_features_addtional_features.head()

,game_time,game_mode,lobby_type,objectives_len,chat_len,number_of_teamfights,r1_kills,r1_deaths,r1_assists,r1_denies,r1_gold,r1_lh,r1_xp,r1_health,r1_max_health,r1_max_mana,r1_level,r1_x,r1_y,r1_stuns,r1_creeps_stacked,r1_camps_stacked,r1_rune_pickups,r1_teamfight_participation,r1_towers_killed,r1_roshans_killed,r1_obs_placed,r1_sen_placed,r1_ability_level,r1_max_hero_hit,r1_purchase_count,r1_buyback,r1_nearby_creep_death_count,r1_damage_deal_toheroes,r1_damage_deal_tocreeps,r1_damage_deal_toneutral,r1_unique_items,r1_number_of_itmes_used,r1_unique_ability,r1_number_of_ability_used,...,r1_deaths_per_minute,r2_deaths_per_minute,r3_deaths_per_minute,r4_deaths_per_minute,r5_deaths_per_minute,d1_deaths_per_minute,d2_deaths_per_minute,d3_deaths_per_minute,d4_deaths_per_minute,d5_deaths_per_minute,r1_number_of_ability_used_per_minute,r2_number_of_ability_used_per_minute,r3_number_of_ability_used_per_minute,r4_number_of_ability_used_per_minute,r5_number_of_ability_used_per_minute,d1_number_of_ability_used_per_minute,d2_number_of_ability_used_per_minute,d3_number_of_ability_used_per_minute,d4_number_of_ability_used_per_minute,d5_number_of_ability_used_per_minute,r1_damage_deal_toheroes_per_minute,r2_damage_deal_toheroes_per_minute,r3_damage_deal_toheroes_per_minute,r4_damage_deal_toheroes_per_minute,r5_damage_deal_toheroes_per_minute,d1_damage_deal_toheroes_per_minute,d2_damage_deal_toheroes_per_minute,d3_damage_deal_toheroes_per_minute,d4_damage_deal_toheroes_per_minute,d5_damage_deal_toheroes_per_minute,r1_damage_deal_toneutral_per_minute,r2_damage_deal_toneutral_per_minute,r3_damage_deal_toneutral_per_minute,r4_damage_deal_toneutral_per_minute,r5_damage_deal_toneutral_per_minute,d1_damage_deal_toneutral_per_minute,d2_damage_deal_toneutral_per_minute,d3_damage_deal_toneutral_per_minute,d4_damage_deal_toneutral_per_minute,d5_damage_deal_toneutral_per_minute
match_id_hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,7,1,11,0,0,0,0,0,543,7,533,358,600,350.938,2,116,122,0.000,0,0,1,0.000,0,0,0,0,0,136,6,0,19,459,828,828,2,2,3,3,...,0.000000,0.000000,0.387097,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.161290,0.774194,0.387097,0.000000,0.387097,0.387097,0.000000,1.161290,1.161290,1.548387,177.677419,0.000000,12.387097,176.903226,0.000000,86.709677,155.612903,245.806452,336.000000,317.419355,320.516129,513.677419,0.000000,210.193548,0.000000,1276.258065,609.677419,43.354839,0.000000,585.290323
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,2,7,2,0,7,5257,52,3937,1160,1160,566.938,8,76,78,0.000,0,0,0,0.438,0,0,0,0,6,216,28,0,79,4656,15306,15306,7,34,2,9,...,0.182371,0.091185,0.091185,0.000000,0.000000,0.182371,0.547112,0.273556,0.273556,0.182371,0.820669,1.914894,1.003040,3.191489,1.641337,1.003040,4.103343,2.006079,1.550152,0.638298,424.559271,464.680851,99.483283,429.300912,174.346505,261.793313,377.598784,187.294833,280.030395,84.437690,1395.683891,725.927052,320.425532,1343.252280,352.887538,295.167173,318.145897,389.726444,195.683891,126.018237
6db558535151ea18ca70a6892197db41,21,23,0,0,0,0,0,0,0,0,176,0,0,680,680,506.938,1,118,118,0.000,0,0,0,0.000,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.857143,0.000000,0.000000,0.000000,2.857143,0.000000,0.000000,0.000000,0.000000,0.000000,200.000000,0.000000,0.000000,0.000000,194.285714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
46a0ddce8f7ed2a8d9bd5edcbb925682,576,22,7,1,4,0,1,0,3,1,1613,0,1471,900,900,290.938,4,170,96,2.366,0,0,5,0.571,0,0,0,0,2,182,17,0,59,3130,220,220,6,10,2,28,...,0.000000,0.000000,0.104167,0.104167,0.000000,0.000000,0.312500,0.104167,0.208333,0.104167,2.916667,5.208333,2.604167,0.729167,0.520833,1.145833,3.229167,1.041667,1.145833,1.979167,326.041667,824.479167,465.416667,73.958333,118.020833,0.000000,217

In [75]:
df_train_features_averages = combine_numeric_features(df_train_features_addtional_features, numeric_features)
df_test_features_averages =  combine_numeric_features(df_test_features_addtional_features, numeric_features)

In [76]:
df_train_features_averages.head()

,game_time,game_mode,lobby_type,objectives_len,chat_len,number_of_teamfights,r_hero_1,r_hero_2,r_hero_3,r_hero_4,r_hero_5,r_hero_6,r_hero_7,r_hero_8,r_hero_9,r_hero_10,r_hero_11,r_hero_12,r_hero_13,r_hero_14,r_hero_15,r_hero_16,r_hero_17,r_hero_18,r_hero_19,r_hero_20,r_hero_21,r_hero_22,r_hero_23,r_hero_25,r_hero_26,r_hero_27,r_hero_28,r_hero_29,r_hero_30,r_hero_31,r_hero_32,r_hero_33,r_hero_34,r_hero_35,...,r_kills_per_minute_sum,d_kills_per_minute_sum,r_xp_per_minute_sum,d_xp_per_minute_sum,r_level_per_minute_sum,d_level_per_minute_sum,r_damage_deal_tocreeps_per_minute_sum,d_damage_deal_tocreeps_per_minute_sum,r_deaths_per_minute_sum,d_deaths_per_minute_sum,r_number_of_ability_used_per_minute_sum,d_number_of_ability_used_per_minute_sum,r_damage_deal_toheroes_per_minute_sum,d_damage_deal_toheroes_per_minute_sum,r_damage_deal_toneutral_per_minute_sum,d_damage_deal_toneutral_per_minute_sum,r_ability_level_sum,d_ability_level_sum,r_max_hero_hit_sum,d_max_hero_hit_sum,r_purchase_count_sum,d_purchase_count_sum,r_damage_deal_toheroes_sum,d_damage_deal_toheroes_sum,r_damage_deal_tocreeps_sum,d_damage_deal_tocreeps_sum,r_unique_items_sum,d_unique_items_sum,r_number_of_itmes_used_sum,d_number_of_itmes_used_sum,r_unique_ability_sum,d_unique_ability_sum,r_number_of_ability_used_sum,d_number_of_ability_used_sum,r_nearby_creep_death_count_sum,d_nearby_creep_death_count_sum,r_buyback_sum,d_buyback_sum,r_damage_deal_toneutral_sum,d_damage_deal_toneutral_sum
match_id_hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,7,1,11,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.387097,770.709677,1193.806452,3.483871,5.032258,1044.387097,2514.580645,0.387097,0.000000,2.709677,4.258065,366.967742,1141.548387,1044.387097,2514.580645,0,0,182,420,33,35,948,2949,2698,6496,11,9,20,12,6,5,7,11,70,104,0,0,2698,6496
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,1.458967,0.273556,1601.762918,1178.571429,3.373860,2.735562,4138.176292,1324.741641,0.364742,1.458967,8.571429,9.300912,1592.370821,1191.155015,4138.176292,1324.741641,28,22,969,764,104,88,17463,13063,45382,14528,36,27,103,67,13,11,94,102,402,255,0,1,45382,14528
6db558535151ea18ca70a6892197db41,21,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,14.285714,14.285714,0.000000,0.000000,0.000000,0.000000,2.857143,2.857143,200.000000,194.285714,0.000000,0.000000,0,0,70,68,21,18,70,68,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0
46a0ddce8f7ed2a8d9bd5edcbb925682,576,22,7,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0.625000,0.104167,1527.395833,1233.645833,3.333333,2.916667,3170.937500,2085.833333,0.208333,0.729167,11.979167,8.541667,1807.916667,868.437500,3170.937500,2085.833333,16,16,773,477,71,73,17356,8337,30441,20024,21,21,62,65,10,16,115,82,367,330,0,0,30441,20024
b1b35ff97723d9b7ade1c9c3cf48f770,453,22,7,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0.264901,0.529801,1292.980132,1431.258278,3.311258,3.443709,4056.688742,4582.384106,0.529801,0.264901,11.523179,9.271523,786.754967,1582.913907,4056.688742,4582.384106,11,11,458,664,43,40,5940,11951,30628,34597,17,14,48,35,12,10,87,70,238,276,0,0,30628,34597


Will add kills/death/assists feature - this one has mentioned in one of the scientific paper about Dota2.

In [77]:
df_train_features_averages["r_kda"] = (df_train_features_averages["r_kills_sum"] + df_train_features_averages['r_assists_sum']) / (df_train_features_averages['r_deaths_sum'] + 1)
df_train_features_averages["d_kda"] = (df_train_features_averages["d_kills_sum"] + df_train_features_averages['d_assists_sum']) / (df_train_features_averages['d_deaths_sum'] + 1)

In [78]:
df_test_features_averages["r_kda"] = (df_test_features_averages["r_kills_sum"] + df_test_features_averages['r_assists_sum']) / (df_test_features_averages['r_deaths_sum'] + 1)
df_test_features_averages["d_kda"] = (df_test_features_averages["d_kills_sum"] + df_test_features_averages['d_assists_sum']) / (df_test_features_averages['d_deaths_sum'] + 1)

In [79]:
df_train_features_diffs = differences(df_train_features_averages,numeric_features,["sum"])
df_test_features_diffs = differences(df_test_features_averages,numeric_features,["sum"])

In [80]:
df_train_features_diffs.head()

,game_time,game_mode,lobby_type,objectives_len,chat_len,number_of_teamfights,r_hero_1,r_hero_2,r_hero_3,r_hero_4,r_hero_5,r_hero_6,r_hero_7,r_hero_8,r_hero_9,r_hero_10,r_hero_11,r_hero_12,r_hero_13,r_hero_14,r_hero_15,r_hero_16,r_hero_17,r_hero_18,r_hero_19,r_hero_20,r_hero_21,r_hero_22,r_hero_23,r_hero_25,r_hero_26,r_hero_27,r_hero_28,r_hero_29,r_hero_30,r_hero_31,r_hero_32,r_hero_33,r_hero_34,r_hero_35,...,health_sum_ratio,gold_sum_ratio,kills_sum_ratio,xp_sum_ratio,x_sum_ratio,y_sum_ratio,max_health_sum_ratio,max_mana_sum_ratio,level_sum_ratio,towers_killed_sum_ratio,stuns_sum_ratio,creeps_stacked_sum_ratio,camps_stacked_sum_ratio,lh_sum_ratio,rune_pickups_sum_ratio,teamfight_participation_sum_ratio,roshans_killed_sum_ratio,obs_placed_sum_ratio,sen_placed_sum_ratio,gold_per_minute_sum_ratio,kills_per_minute_sum_ratio,xp_per_minute_sum_ratio,level_per_minute_sum_ratio,damage_deal_tocreeps_per_minute_sum_ratio,deaths_per_minute_sum_ratio,number_of_ability_used_per_minute_sum_ratio,damage_deal_toheroes_per_minute_sum_ratio,damage_deal_toneutral_per_minute_sum_ratio,ability_level_sum_ratio,max_hero_hit_sum_ratio,purchase_count_sum_ratio,damage_deal_toheroes_sum_ratio,damage_deal_tocreeps_sum_ratio,unique_items_sum_ratio,number_of_itmes_used_sum_ratio,unique_ability_sum_ratio,number_of_ability_used_sum_ratio,nearby_creep_death_count_sum_ratio,buyback_sum_ratio,damage_deal_toneutral_sum_ratio
match_id_hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,7,1,11,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.747844,0.515957,0.000000,0.645590,0.917431,0.948553,1.000000,0.765756,0.692308,NaN,0.000000,NaN,NaN,0.781250,0.285714,0.000000,NaN,0.000000,NaN,0.515957,0.000000,0.645590,0.692308,0.415333,inf,0.636364,0.321465,0.415333,NaN,0.433333,0.942857,0.321465,0.415333,1.222222,1.666667,1.200000,0.636364,0.673077,NaN,0.415333
b9c57c450ce74a2af79c9ce96fac144d,658,4,0,3,10,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,1.358912,1.382532,5.333333,1.359072,0.913690,0.939726,1.151111,1.086382,1.233333,inf,11.904897,0.666667,0.5,1.322917,0.928571,1.250000,NaN,0.800000,1.5,1.382532,5.333333,1.359072,1.233333,3.123761,0.250000,0.921569,1.336829,3.123761,1.272727,1.268325,1.181818,1.336829,3.123761,1.333333,1.537313,1.181818,0.921569,1.576471,0.0,3.123761
6db558535151ea18ca70a6892197db41,21,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.994278,1.833333,NaN,NaN,0.728643,0.796196,0.993711,1.052940,1.000000,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,1.833333,NaN,NaN,1.000000,NaN,NaN,1.000000,1.029412,NaN,NaN,1.029412,1.166667,1.029412,NaN,0.000000,0.000000,1.000000,1.000000,NaN,NaN,NaN
46a0ddce8f7ed2a8d9bd5edcbb925682,576,22,7,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,1.074394,1.181134,6.000000,1.238115,1.212230,0.847222,0.983333,1.176916,1.142857,NaN,0.834090,0.000000,0.0,0.850649,1.181818,2.000000,NaN,1.333333,NaN,1.181134,6.000000,1.238115,1.142857,1.520226,0.285714,1.402439,2.081804,1.520226,1.000000,1.620545,0.972603,2.081804,1.520226,1.000000,0.953846,0.625000,1.402439,1.112121,NaN,1.520226
b1b35ff97723d9b7ade1c9c3cf48f770,453,22,7,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,1.115213,0.880353,0.500000,0.903387,0.885886,0.961310,1.194872,1.207281,0.961538,NaN,0.568942,0.000000,0.0,0.613861,1.375000,1.666667,NaN,NaN,NaN,0.880353,0.500000,0.903387,0.961538,0.885279,2.000000,1.242857,0.497030,0.885279,1.000000,0.689759,1.075000,0.497030,0.885279,1.214286,1.371429,1.200000,1.242857,0.862319,NaN,0.885279


Now we do our final steps before modeling - i have excluded features that seems like irrelevant for win/loose prediction.

In [81]:
cols_to_drop = [ 'game_mode','lobby_type','objectives_len','chat_len',
                'x_sum_ratio','y_sum_ratio', 'obs_placed_sum_ratio',
                'sen_placed_sum_ratio', 'max_health_sum_ratio','max_mana_sum_ratio','lh_sum_ratio',
                "max_hero_hit_sum_ratio","unique_items_sum_ratio","unique_ability_sum_ratio"]

In [82]:
df_train_features_diffs.drop(cols_to_drop, axis = 1,inplace = True)
df_test_features_diffs.drop(cols_to_drop, axis = 1,inplace = True)

Merge main and teamfight data together and create time-based features.

In [83]:
#Merge teamfight data 
df_final =df_train_features_diffs.merge(df_train_teamfight_features_average, how='left', left_index=True, right_index=True )
df_test_final =df_test_features_diffs.merge(test_teamfight_new_features_average, how='left', left_index=True, right_index=True)


df_final["short"] = df_final["game_time"].apply(lambda x : x <= 300)
df_test_final["short"] = df_test_final["game_time"].apply(lambda x : x <= 300)

df_final["middle"] = df_final["game_time"].apply(lambda x : (x > 300) & (x  < 1200))
df_test_final["middle"] = df_test_final["game_time"].apply(lambda x : (x > 300) & (x  < 1200))

df_final["long"] = df_final["game_time"].apply(lambda x : x >= 1200)
df_test_final["long"] = df_test_final["game_time"].apply(lambda x : x >= 1200)

df_final["any_fight"] = df_final["number_of_teamfights"].apply(lambda x : x > 0)
df_test_final["any_fight"] = df_test_final["number_of_teamfights"].apply(lambda x : x > 0)

Perform simple modeling using LightGBM - get params by random search on the grid (in another script) and use 5 - fold cv scheme with early stopping.

In [84]:
df_final.head()

,game_time,number_of_teamfights,r_hero_1,r_hero_2,r_hero_3,r_hero_4,r_hero_5,r_hero_6,r_hero_7,r_hero_8,r_hero_9,r_hero_10,r_hero_11,r_hero_12,r_hero_13,r_hero_14,r_hero_15,r_hero_16,r_hero_17,r_hero_18,r_hero_19,r_hero_20,r_hero_21,r_hero_22,r_hero_23,r_hero_25,r_hero_26,r_hero_27,r_hero_28,r_hero_29,r_hero_30,r_hero_31,r_hero_32,r_hero_33,r_hero_34,r_hero_35,r_hero_36,r_hero_37,r_hero_38,r_hero_39,...,health_sum_ratio,gold_sum_ratio,kills_sum_ratio,xp_sum_ratio,level_sum_ratio,towers_killed_sum_ratio,stuns_sum_ratio,creeps_stacked_sum_ratio,camps_stacked_sum_ratio,rune_pickups_sum_ratio,teamfight_participation_sum_ratio,roshans_killed_sum_ratio,gold_per_minute_sum_ratio,kills_per_minute_sum_ratio,xp_per_minute_sum_ratio,level_per_minute_sum_ratio,damage_deal_tocreeps_per_minute_sum_ratio,deaths_per_minute_sum_ratio,number_of_ability_used_per_minute_sum_ratio,damage_deal_toheroes_per_minute_sum_ratio,damage_deal_toneutral_per_minute_sum_ratio,ability_level_sum_ratio,purchase_count_sum_ratio,damage_deal_toheroes_sum_ratio,damage_deal_tocreeps_sum_ratio,number_of_itmes_used_sum_ratio,number_of_ability_used_sum_ratio,nearby_creep_death_count_sum_ratio,buyback_sum_ratio,damage_deal_toneutral_sum_ratio,r_teamfight_damage_sum,d_teamfight_damage_sum,r_teamfight_gold_delta_sum,d_teamfight_gold_delta_sum,r_teamfight_xp_delta_sum,d_teamfight_xp_delta_sum,short,middle,long,any_fight
match_id_hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.747844,0.515957,0.000000,0.645590,0.692308,NaN,0.000000,NaN,NaN,0.285714,0.000000,NaN,0.515957,0.000000,0.645590,0.692308,0.415333,inf,0.636364,0.321465,0.415333,NaN,0.942857,0.321465,0.415333,1.666667,0.636364,0.673077,NaN,0.415333,0.0,0.0,0.0,0.0,0.0,0.0,True,False,False,False
b9c57c450ce74a2af79c9ce96fac144d,658,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,1.358912,1.382532,5.333333,1.359072,1.233333,inf,11.904897,0.666667,0.5,0.928571,1.250000,NaN,1.382532,5.333333,1.359072,1.233333,3.123761,0.250000,0.921569,1.336829,3.123761,1.272727,1.181818,1.336829,3.123761,1.537313,0.921569,1.576471,0.0,3.123761,4848.0,3081.0,2435.0,1696.0,2273.0,514.0,False,True,False,True
6db558535151ea18ca70a6892197db41,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.994278,1.833333,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,1.833333,NaN,NaN,1.000000,NaN,NaN,1.000000,1.029412,NaN,NaN,1.166667,1.029412,NaN,0.000000,1.000000,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,True,False,False,False
46a0ddce8f7ed2a8d9bd5edcbb925682,576,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1.074394,1.181134,6.000000,1.238115,1.142857,NaN,0.834090,0.000000,0.0,1.181818,2.000000,NaN,1.181134,6.000000,1.238115,1.142857,1.520226,0.285714,1.402439,2.081804,1.520226,1.000000,0.972603,2.081804,1.520226,0.953846,1.402439,1.112121,NaN,1.520226,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False,False
b1b35ff97723d9b7ade1c9c3cf48f770,453,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,1.115213,0.880353,0.500000,0.903387,0.961538,NaN,0.568942,0.000000,0.0,1.375000,1.666667,NaN,0.880353,0.500000,0.903387,0.961538,0.885279,2.000000,1.242857,0.497030,0.885279,1.000000,1.075000,0.497030,0.885279,1.371429,1.242857,0.862319,NaN,0.885279,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False,False


In [85]:
import lightgbm as  lgb
from sklearn.model_selection import KFold
skf = KFold(n_splits=5, random_state=17)

In [86]:

params = {
'num_leaves': 24,
 'reg_alpha': 0.9387755102040816,
 'subsample_for_bin': 80000,
 'colsample_bytree': 0.6,
 'learning_rate': 0.007364413619537508,
 'subsample': 0.98989898989899, 
 'reg_lambda': 0.061224489795918366,
 'is_unbalance': True,
 'min_child_samples': 495,
 'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc'
         }

In [88]:
from sklearn.metrics import roc_auc_score
predict = [0]*df_test_final.shape[0]
cross_val_result = 0
for ind_trn, ind_test in skf.split(df_final, target_variable): 
    
   
    
    X_train_lgbm = lgb.Dataset(df_final.iloc[ind_trn], target_variable.iloc[ind_trn] )
    X_test_lgbm = lgb.Dataset(df_final.iloc[ind_test], target_variable.iloc[ind_test])
    
    
    clf = lgb.train(params, X_train_lgbm, num_boost_round = 10000, valid_sets=X_test_lgbm, early_stopping_rounds=100, verbose_eval=200 )
    
    cross_val_predict = clf.predict(df_final.iloc[ind_test])
    cross_val_auc = roc_auc_score(target_variable.iloc[ind_test] , cross_val_predict) / 5
    
    cross_val_result +=cross_val_auc
    
    pred = clf.predict(df_test_final)/5
    predict += pred  
    
print(cross_val_result)

Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.824075
[400]	valid_0's auc: 0.828624
[600]	valid_0's auc: 0.833777
[800]	valid_0's auc: 0.836712
[1000]	valid_0's auc: 0.838663
[1200]	valid_0's auc: 0.839941
[1400]	valid_0's auc: 0.840779
[1600]	valid_0's auc: 0.841335
[1800]	valid_0's auc: 0.84173
[2000]	valid_0's auc: 0.841924
Early stopping, best iteration is:
[2060]	valid_0's auc: 0.841961
Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.821618
[400]	valid_0's auc: 0.828384
[600]	valid_0's auc: 0.833341
[800]	valid_0's auc: 0.836583
[1000]	valid_0's auc: 0.838655
[1200]	valid_0's auc: 0.840035
[1400]	valid_0's auc: 0.841003
[1600]	valid_0's auc: 0.841612
[1800]	valid_0's auc: 0.842036
[2000]	valid_0's auc: 0.842384
[2200]	valid_0's auc: 0.842507
[2400]	valid_0's auc: 0.842657
Early stopping, best iteration is:
[2418]	valid_0's auc: 0.842668
Training until validation scores don't improve for 100 rounds.
[200]	

In [ ]:
df_submission = pd.DataFrame({'radiant_win_prob': predict}, 
                                 index=df_test_features.index)
df_submission.to_csv("LGB_with_heroesdummy_scaling_averages_diffs_03_04.csv")